## Generates the poule sheets for Judo+ tournaments  
#### Work by [Gonçalo Sousa](https://github.com/Sousa1909) for [AJDS](https://github.com/AJD-Santarem)  

---

In [43]:
import pandas as pd
from openpyxl import load_workbook
import os
import yaml
import random

# Create Input Folder if it doesn't exist
if not os.path.exists("Input"):
    os.makedirs("Input")
    print("Input folder does not exist. Generating...")

Table Manipulation

In [44]:
# Read the Excel file (Masculine and Feminine sheets) without skipping rows
df_masc = pd.read_excel('Input/judo+_test.xlsx', sheet_name='MASC')
df_fem = pd.read_excel('Input/judo+_test.xlsx', sheet_name='FEM')

# Names for the output PDFs
tournament_code = "judo_plus_test"

# Function to process the data (drop the first row, set column names, and sort)
def process_table(df):
    # Drop the first row ( TITLE )
    df = df.drop(index=[0])

    # Set new column names
    df.columns = ['NAME', 'AGE_TIER', 'WEIGHT', 'CLUB']

    # Sort by CLUB (ascending) and WEIGHT (ascending)
    df = df.sort_values(by=['AGE_TIER', 'WEIGHT'], ascending=[True, True])

    return df

# Process both tables
df_masc = process_table(df_masc)
df_fem = process_table(df_fem)

Athlete Grouping and Draw

In [45]:
# Function to check if a group has an athlete from the same club
def has_same_club(group, club):
    return any(athlete['CLUB'] == club for athlete in group)

# Function to split a list into groups of 5, 4, or 3, ideally
def divide_into_groups(athletes, column_order, target_group_sizes=[4, 3]):
    random.shuffle(athletes)  # Shuffle athletes randomly
    result_groups = []
    
    # Split into groups of the preferred sizes (4 or 3)
    i = 0
    while i < len(athletes):
        remaining = len(athletes) - i
        
        # Try to form a group of 4 or 3 based on the remaining number of athletes
        for size in target_group_sizes:
            if remaining >= size or remaining == 2:  # If we're down to 2 athletes, group them as 2
                group = []
                for athlete in athletes[i:i + size]:
                    # Check if the athlete's club is already in the group, and avoid if possible
                    if has_same_club(group, athlete['CLUB']):
                        # Try to add this athlete to a different group, but only if groups exist
                        added = False
                        if result_groups:
                            for prev_group in result_groups:
                                if len(prev_group) < 5 and not has_same_club(prev_group.to_dict(orient='records'), athlete['CLUB']):
                                    prev_group = pd.concat([prev_group, pd.DataFrame([athlete])])
                                    added = True
                                    break
                        if not added:
                            group.append(athlete)
                    else:
                        group.append(athlete)
                group_df = pd.DataFrame(group)  # Convert to DataFrame
                result_groups.append(group_df[column_order])  # Reorder columns consistently
                i += size
                break
        else:
            # Handle the case where we have one athlete left
            group = athletes[i:]
            group_df = pd.DataFrame(group)
            # Save this group as a "lone" athlete
            result_groups.append(group_df[column_order])
            break

    # Handle the case of a lone athlete by merging them with another group
    if len(result_groups[-1]) == 1:
        lone_athlete = result_groups.pop()  # Remove the lone athlete's group
        for group in result_groups:
            if len(group) < 5:  # Add them to a group with less than 5 members
                result_groups[result_groups.index(group)] = pd.concat([group, lone_athlete], ignore_index=True)
                break

    return result_groups

# Function to group athletes by 4kg weight ranges
def group_by_weight_and_divide(df, weight_range):
    column_order=['NAME', 'WEIGHT', 'CLUB']
    df = df.sort_values(by='WEIGHT').reset_index(drop=True)  # Sort by weight
    groups = []
    current_group = []
    current_limit = df.iloc[0]['WEIGHT'] + weight_range  # Set initial weight range limit
    
    for _, row in df.iterrows():
        if row['WEIGHT'] <= current_limit:
            current_group.append(row)
        else:
            groups.append(current_group)  # Save the group
            current_group = [row]  # Start a new group
            current_limit = row['WEIGHT'] + weight_range  # Set new weight limit for the next group
    
    groups.append(current_group)  # Append the last group

    # Divide each weight group into subgroups of 5, 4, or 3
    final_groups = []
    for group in groups:
        athlete_list = [row for row in group]
        divided_groups = divide_into_groups(athlete_list, column_order=column_order)
        final_groups.extend(divided_groups)

    return final_groups

# Group athletes by AGE_TIER first, and then by weight
def group_by_age_and_weight(df, weight_range=4.0):
    grouped_by_age_tier = []
    
    # Group athletes by 'AGE_TIER'
    for age_tier, group in df.groupby('AGE_TIER'):
        print(f"Grouping athletes in {age_tier}...")
        
        # Sort by weight and apply the weight-based grouping within each AGE_TIER
        groups_by_weight = group_by_weight_and_divide(group, weight_range)
        
        # Store the grouped athletes with their respective age_tier label
        grouped_by_age_tier.append((age_tier, groups_by_weight))
    
    return grouped_by_age_tier

# To create a folder for each tournament
output_folder = "Output/" + tournament_code

# Function thats saves the groups in YAML files
def save_grouped_athletes_to_yaml(groups_by_age, gender):
    os.makedirs(output_folder, exist_ok=True)
    
    output_yaml_file = f'{output_folder}/.grouped_athletes_{gender}.yaml'
    
    # Structure to hold the grouped athletes for YAML
    grouped_data = {}

    for age_tier, groups in groups_by_age:
        grouped_data[age_tier] = []  # Each age tier will have its own list of groups
        for i, group in enumerate(groups):
            group_dict = {
                f'Group {i + 1}': [
                    {'Name': athlete.NAME, 'Weight': athlete.WEIGHT, 'Club': athlete.CLUB}
                    for athlete in group.itertuples(index=False)
                ]
            }
            grouped_data[age_tier].append(group_dict)  # Append the group data under the correct age tier

    # Save the structured data as YAML
    with open(output_yaml_file, 'w', encoding='utf-8') as f:
        yaml.dump(grouped_data, f, allow_unicode=True, sort_keys=False)

    print(f"Grouped athletes for {gender} have been saved to {output_yaml_file}.")


# Group athletes by AGE_TIER and weight for both genders
groups_fem = group_by_age_and_weight(df_fem)
save_grouped_athletes_to_yaml(groups_fem, 'fem')

groups_mas = group_by_age_and_weight(df_masc)
save_grouped_athletes_to_yaml(groups_mas, 'mas')

Grouping athletes in Benjamim...
Grouping athletes in Infantil...
Grouping athletes in Iniciada...
Grouped athletes for fem have been saved to Output/judo_plus_test/.grouped_athletes_fem.yaml.
Grouping athletes in Bejnamim...
Grouping athletes in Benjamim...
Grouping athletes in Infantil...
Grouping athletes in Iniciado...
Grouped athletes for mas have been saved to Output/judo_plus_test/.grouped_athletes_mas.yaml.


In [46]:
best_of_3 = "Templates/BO_3.xlsx"
poule_of_5 ="Templates/Poule_5.xlsx"

# Load the appropriate template based on group size
def load_template(group_size, template_file_5, template_file_2):
    if group_size == 2:
        return True, load_workbook(template_file_2)
    else:
        return False,load_workbook(template_file_5)

# Populate the table with the group information
def populate_group_table(group, template_file_5, template_file_2, output_file, group_number, age_tier):
    # Load the correct template based on the group size
    bo3, workbook = load_template(len(group), template_file_5, template_file_2)
    sheet = workbook.active

    # Fill in athlete data
    if bo3:
        for i, athlete in enumerate(group.itertuples(), start=6):
            sheet[f'C{i + 2}'] = athlete.NAME
            sheet[f'D{i + 2}'] = athlete.CLUB
        sheet[f'F{13}'] = group_number
        sheet[f'K{4}'] = age_tier
    else:
        for i, athlete in enumerate(group.itertuples(), start=7):
            sheet[f'B{i + 2}'] = athlete.NAME
            sheet[f'C{i + 2}'] = athlete.CLUB
        sheet[f'L{17}'] = group_number
        sheet[f'M{4}'] = age_tier

    # Save the updated file to the designated folder
    output_filename = os.path.join(output_folder, output_file.format(age_tier, group_number))
    workbook.save(output_filename)
    #print(f"Group {group_number} saved to {output_filename}")

# Function to process each group and generate tables
def create_tables_for_groups(groups_by_age_tier, template_file_5, template_file_2, output_file_pattern):
    for age_tier, groups in groups_by_age_tier:  # Expecting a tuple of (age_tier, groups)
        group_number = 1  # To keep track of the group number across all age tiers
        for group in groups:  # For each weight group in this age tier
            populate_group_table(group, template_file_5, template_file_2, output_file_pattern, group_number, age_tier)
            group_number += 1

create_tables_for_groups(groups_mas, poule_of_5, best_of_3, 'Group_M_{}_{}.xlsx')
create_tables_for_groups(groups_fem, poule_of_5, best_of_3, 'Group_F_{}_{}.xlsx')